In [1]:
import findspark
findspark.init('C:\spark\spark-3.1.2-bin-hadoop3.2')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lreg').getOrCreate()
spark

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [14]:
training.head()

Row(label=-9.490009878824548, features=SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658}))

In [15]:
lr = LinearRegression(featuresCol='features', labelCol='label',
                     predictionCol='prediction')

In [16]:
lrModel = lr.fit(training)

In [17]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [18]:
len(lrModel.coefficients)

10

In [19]:
lrModel.intercept

0.14228558260358093

In [20]:
training_summary = lrModel.summary

In [24]:
lrModel.summary.rootMeanSquaredError

10.16309157133015

In [25]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [26]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [27]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
+----------

In [28]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 342|
|   mean|-0.06515885592583089|
| stddev|  10.446014641623218|
|    min| -28.571478869743427|
|    max|  27.111027963108548|
+-------+--------------------+



In [29]:
correct_model = lr.fit(train_data)

In [31]:
test_results = correct_model.evaluate(test_data)

In [32]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -27.78527751960659|
|-26.773666161867546|
|-22.024337370690972|
|-14.774270328922197|
| -17.56033588373536|
| -20.31910195808907|
|-16.495645329161558|
|-16.130575589643495|
|-13.934980593936528|
|-17.132703793727014|
|-12.245551702417917|
| -16.49753993214495|
| -11.71866423441971|
| -12.93309366423493|
|-15.750008747028925|
| -13.99088365396046|
| -9.453827478590826|
|-12.204070213712088|
| -12.05833911977211|
|  -9.95519536214369|
+-------------------+
only showing top 20 rows



In [33]:
unlabeled_data = test_data.select('features')

In [34]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [36]:
predictions = correct_model.transform(unlabeled_data)

In [37]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.2607405181700432|
|(10,[0,1,2,3,4,5,...|   3.823840225671471|
|(10,[0,1,2,3,4,5,...|  0.5919496065251669|
|(10,[0,1,2,3,4,5,...|  -4.394022294041564|
|(10,[0,1,2,3,4,5,...| -0.7148776822692717|
|(10,[0,1,2,3,4,5,...|  3.6268949367779673|
|(10,[0,1,2,3,4,5,...|  0.3442959778844463|
|(10,[0,1,2,3,4,5,...| 0.04491654862200503|
|(10,[0,1,2,3,4,5,...| -1.9270287336340324|
|(10,[0,1,2,3,4,5,...|  1.7731589138943369|
|(10,[0,1,2,3,4,5,...|  -3.103319452961337|
|(10,[0,1,2,3,4,5,...|  2.1685614230695083|
|(10,[0,1,2,3,4,5,...| -1.7019305414710473|
|(10,[0,1,2,3,4,5,...|-0.10683439986968532|
|(10,[0,1,2,3,4,5,...|  3.2492349616738716|
|(10,[0,1,2,3,4,5,...|  1.5116034425089613|
|(10,[0,1,2,3,4,5,...| -2.6765257336971033|
|(10,[0,1,2,3,4,5,...| 0.10971893517682876|
|(10,[0,1,2,3,4,5,...| 0.15335221709699684|
|(10,[0,1,2,3,4,5,...| -0.533961

In [51]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True, header=True)

In [52]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [53]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [57]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [58]:
for i in data.head():
    print(i)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [59]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [60]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [61]:
assembler = VectorAssembler(inputCols= ['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],
                           outputCol='features')

In [62]:
output = assembler.transform(data)

In [63]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [64]:
output.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent',
 'features']

In [65]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [66]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [67]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [68]:
final_data = output.select('features', 'Yearly Amount Spent')

In [69]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [71]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [72]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                329|
|   mean|   498.427973271839|
| stddev|  76.40015056981278|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [73]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                171|
|   mean|  501.0188065664343|
| stddev|  84.84729388339491|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [74]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [75]:
lr_model = lr.fit(train_data)

In [76]:
test_results = lr_model.evaluate(test_data)

In [78]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  10.67616144791441|
| -5.393924888019512|
| 0.3088805072372338|
| 11.007372080044092|
|-17.433790323139533|
|   7.14954468968449|
| -5.727830014050653|
|-12.759267268959036|
| -6.810957126335325|
|   10.3076462642764|
| 22.801118304877264|
| 18.652810168658846|
|  4.283316778100584|
| -5.360590828336456|
|  -9.33780247574822|
|-17.071411286985494|
| 17.106068159248082|
|  -4.80632437786204|
|-1.1328595256881044|
|-10.924123304974728|
+-------------------+
only showing top 20 rows



In [79]:
test_results.rootMeanSquaredError

9.953634871208576

In [80]:
test_results.r2

0.9861568600662352